In [100]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st
import spotipy 

# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Importar funciones de gestión
from utils.chat_gpt_manager import (
    get_instruments_chat,
    get_chords_chat
)
from utils.spotify_manager import (
    get_similar_songs,
    get_track_id_spotify,
    get_audio_analysis,
    get_audio_features,
    get_song_and_artist
)
from utils.functions import (
    add_info_list_df,
    convert_chords_with_minor,
    standardize_chord_symbols,
    transpose_chords_to_key,
    check_track_csv,
    clean_metadata,
    filter_existing_vector_id,
    to_embbed_text_column,
    expand_and_remove_original_columns,
    drop_columns_with_many_nulls,
    get_user_input,
    handle_input_search,
    calc_diff,
    export_unique_rows,
    create_metadata_column,
    clean_metadata_column,
    insert_into_pinecone,
    create_dataframe_for_pinecone,
    realizar_consulta

    
)

from utils.pinecone_manager import (
    initialize_pinecone, 
    setup_index
)

from utils.lyrics_manager import insert_lyrics_db

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Initizalize pinecone

In [101]:
pc = initialize_pinecone()

In [102]:
index = setup_index(pc)

Index 'song-starter-index' already exists, skipping creation.


In [103]:
index_name= pc.list_indexes().indexes[0].name

# API de Spotify

Aquí se utiliza la API de Spotify para obtener recomendaciones de canciones basadas en una canción dada. 

Input:

In [104]:
input_song, input_artist = get_user_input()

In [105]:
# Llamar a la función para obtener el track ID
input_track_id=handle_input_search(input_song, input_artist)

In [106]:
input_song , input_artist = get_song_and_artist(input_track_id)

In [107]:
input_data = pd.DataFrame({'id': [input_track_id]})
input_data['name'] = input_song
input_data['artists'] = input_artist

In [108]:
input_data

,id,name,artists
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo


Similar songs

In [109]:
tracks_data = get_similar_songs(input_track_id, 5)

In [110]:
tracks_data = pd.DataFrame(tracks_data)

In [111]:
tracks_data.head(2)

,id,name,artists
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo


MIRAR ESTO, cómo evitar que a partir de aquí todo de error si ya están todas en pinecone

In [112]:
filter_existing_vector_id(index, tracks_data)            

In [113]:

tracks_data.reset_index(drop=True, inplace=True)

In [114]:
tracks_data

,id,name,artists
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez


In [115]:
csv= pd.read_csv('mi_archivo_temporal.csv')

In [116]:
if len(tracks_data)>0:
    df=check_track_csv(tracks_data, csv)

In [117]:
if len(df)>0:    
    df.reset_index(drop=True, inplace=True)

# API de ChatGPT

Aquí se utiliza la API de Chat GPT, para obtener los acordes e instrumentos de las canciones

Input:

In [118]:
add_info_list_df(input_data, get_chords_chat, 'chords')

,id,name,artists,chords
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]"


In [119]:
add_info_list_df(input_data, get_instruments_chat, 'instruments')

,id,name,artists,chords,instruments
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]"


Related tracks:

In [120]:
add_info_list_df(df, get_chords_chat, 'chords')

,id,name,artists,chords
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[G, D, Em, C, G, D, C, G]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[D, A, Bm, G, D, A, G, A]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[F, G, Am, Em, F, G, C, Am]"
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,"[A, C#m, F#m, E, A, C#m, F#m, E]"
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,"[Am, D7, G, C, F, E7, Am, G]"


In [121]:
add_info_list_df(df, get_instruments_chat, 'instruments')

,id,name,artists,chords,instruments
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[G, D, Em, C, G, D, C, G]","[guitar, accordion]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[D, A, Bm, G, D, A, G, A]","[guitar, bass]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[F, G, Am, Em, F, G, C, Am]","[guitar, drums]"
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,"[A, C#m, F#m, E, A, C#m, F#m, E]","[guitar, bass]"
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,"[Am, D7, G, C, F, E7, Am, G]","[guitar, piano]"


# API de Spotify para nuevas columnas

Input:

In [122]:
get_audio_features(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055


In [123]:
get_audio_analysis(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068


In [124]:
insert_lyrics_db(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca\nCora...


In [125]:
input_data['chords'].dropna(inplace=True)
input_data = input_data[input_data['chords'].apply(lambda x: x != [])]

In [126]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca\nCora...


Related tracks:

In [127]:
get_audio_features(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[G, D, Em, C, G, D, C, G]","[guitar, accordion]",0.0541,0.000000
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[D, A, Bm, G, D, A, G, A]","[guitar, bass]",0.0368,0.000000
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[F, G, Am, Em, F, G, C, Am]","[guitar, drums]",0.0450,0.014100
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,"[A, C#m, F#m, E, A, C#m, F#m, E]","[guitar, bass]",0.0282,0.000012
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,"[Am, D7, G, C, F, E7, Am, G]","[guitar, piano]",0.0841,0.000001


In [128]:
get_audio_analysis(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[G, D, Em, C, G, D, C, G]","[guitar, accordion]",0.0541,0.000000,0,1,102.009,-7.864
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[D, A, Bm, G, D, A, G, A]","[guitar, bass]",0.0368,0.000000,7,1,79.923,-6.206
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[F, G, Am, Em, F, G, C, Am]","[guitar, drums]",0.0450,0.014100,4,0,107.069,-6.422
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,"[A, C#m, F#m, E, A, C#m, F#m, E]","[guitar, bass]",0.0282,0.000012,8,0,129.881,-6.968
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,"[Am, D7, G, C, F, E7, Am, G]","[guitar, piano]",0.0841,0.000001,8,1,152.197,-12.273


In [129]:
insert_lyrics_db(df)

Timeout o fallo! Usando GPT para obtener el tema para La Ley - El Duelo
Timeout o fallo! Usando GPT para obtener el tema para Silvio Rodríguez - Canción Del Elegido


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[G, D, Em, C, G, D, C, G]","[guitar, accordion]",0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino\nYo te...
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[D, A, Bm, G, D, A, G, A]","[guitar, bass]",0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[F, G, Am, Em, F, G, C, Am]","[guitar, drums]",0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro..."
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,"[A, C#m, F#m, E, A, C#m, F#m, E]","[guitar, bass]",0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide\ntodo...
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,"[Am, D7, G, C, F, E7, Am, G]","[guitar, piano]",0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl..."


In [130]:
df_vectorial= df.copy()
df_vectorial['chords'].dropna(inplace=True)
df_vectorial = df_vectorial[df_vectorial['chords'].apply(lambda x: x != [])]

# Separate columns

In [131]:
chord_map = {
    'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3, 'E': 4, 
    'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8, 'Ab': 8, 'A': 9, 
    'A#': 10, 'Bb': 10, 'B': 11
}

Input:

In [132]:
input_data['chords_1'] = input_data['chords'].apply(lambda x: x[:4]) 
input_data['chords_2'] = input_data['chords'].apply(lambda x: x[4:])

In [133]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca\nCora...,"[A, D, E, A]","[A, D, E, F#m]"


In [134]:
input_data['chords_1'] = input_data['chords_1'].apply(standardize_chord_symbols)
input_data['chords_2'] = input_data['chords_2'].apply(standardize_chord_symbols)

In [135]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca\nCora...,"[A, D, E, A]","[A, D, E, F#m]"


In [136]:
input_data['chords_1_numeric'] = input_data['chords_1'].apply(convert_chords_with_minor)
input_data['chords_2_numeric'] = input_data['chords_2'].apply(convert_chords_with_minor)

In [137]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca\nCora...,"[A, D, E, A]","[A, D, E, F#m]","[9, 2, 4, 9]","[9, 2, 4, 18]"


In [138]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    input_data[f'normal_{col}'] = input_data.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca\nCora...,"[A, D, E, A]","[A, D, E, F#m]","[9, 2, 4, 9]","[9, 2, 4, 18]","[2, -5, -3, 2]","[2, -5, -3, 11]"


In [139]:
for col in col_chords:
    input_data[f'diff_{col}'] = input_data.apply(lambda row: calc_diff(row[col]), axis=1)

input_data


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca\nCora...,"[A, D, E, A]","[A, D, E, F#m]","[9, 2, 4, 9]","[9, 2, 4, 18]","[2, -5, -3, 2]","[2, -5, -3, 11]","[-7, 2, 5]","[-7, 2, 14]"


In [140]:
input_data.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [141]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca Coral...,"[A, D, E, A]","[A, D, E, F#m]","[9, 2, 4, 9]","[9, 2, 4, 18]","[2, -5, -3, 2]","[2, -5, -3, 11]","[-7, 2, 5]","[-7, 2, 14]"


Related tracks:

In [142]:
df_vectorial['chords_1'] = df_vectorial['chords'].apply(lambda x: x[:4]) 
df_vectorial['chords_2'] = df_vectorial['chords'].apply(lambda x: x[4:])
df_vectorial.drop(columns=['chords'], inplace=True)

In [143]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[guitar, accordion]",0.0541,0.0000,0,1,102.009,-7.864,Te mando flores que recojo en el camino\nYo te...,"[G, D, Em, C]","[G, D, C, G]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[guitar, bass]",0.0368,0.0000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,"[D, A, Bm, G]","[D, A, G, A]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[guitar, drums]",0.0450,0.0141,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...","[F, G, Am, Em]","[F, G, C, Am]"


In [144]:
# Nos ocupamos de bemoles o sostenidos para estandarizarlos
df_vectorial['chords_1'] = df_vectorial['chords_1'].apply(standardize_chord_symbols)
df_vectorial['chords_2'] = df_vectorial['chords_2'].apply(standardize_chord_symbols)

In [145]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[guitar, accordion]",0.0541,0.0000,0,1,102.009,-7.864,Te mando flores que recojo en el camino\nYo te...,"[G, D, Em, C]","[G, D, C, G]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[guitar, bass]",0.0368,0.0000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,"[D, A, Bm, G]","[D, A, G, A]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[guitar, drums]",0.0450,0.0141,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...","[F, G, Am, Em]","[F, G, C, Am]"


In [146]:
# Convertimos los acordes a valores numéricos
df_vectorial['chords_1_numeric'] = df_vectorial['chords_1'].apply(convert_chords_with_minor)
df_vectorial['chords_2_numeric'] = df_vectorial['chords_2'].apply(convert_chords_with_minor)

In [147]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[guitar, accordion]",0.0541,0.0000,0,1,102.009,-7.864,Te mando flores que recojo en el camino\nYo te...,"[G, D, Em, C]","[G, D, C, G]","[7, 2, 16, 0]","[7, 2, 0, 7]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[guitar, bass]",0.0368,0.0000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,"[D, A, Bm, G]","[D, A, G, A]","[2, 9, 23, 7]","[2, 9, 7, 9]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[guitar, drums]",0.0450,0.0141,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...","[F, G, Am, Em]","[F, G, C, Am]","[5, 7, 21, 16]","[5, 7, 0, 21]"


In [148]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    df_vectorial[f'normal_{col}'] = df_vectorial.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[guitar, accordion]",0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino\nYo te...,"[G, D, Em, C]","[G, D, C, G]","[7, 2, 16, 0]","[7, 2, 0, 7]","[7, 2, 16, 0]","[7, 2, 0, 7]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[guitar, bass]",0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,"[D, A, Bm, G]","[D, A, G, A]","[2, 9, 23, 7]","[2, 9, 7, 9]","[-5, 2, 16, 0]","[-5, 2, 0, 2]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[guitar, drums]",0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...","[F, G, Am, Em]","[F, G, C, Am]","[5, 7, 21, 16]","[5, 7, 0, 21]","[1, 3, 17, 12]","[1, 3, -4, 17]"
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,"[guitar, bass]",0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide\ntodo...,"[A, C#m, F#m, E]","[A, C#m, F#m, E]","[9, 13, 18, 4]","[9, 13, 18, 4]","[1, 5, 10, -4]","[1, 5, 10, -4]"
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,"[guitar, piano]",0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl...","[Am, D7, G, C]","[F, E7, Am, G]","[21, 2, 7, 0]","[5, 4, 21, 7]","[13, -6, -1, -8]","[-3, -4, 13, -1]"


In [149]:
for col in col_chords:
    df_vectorial[f'diff_{col}'] = df_vectorial.apply(lambda row: calc_diff(row[col]), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[guitar, accordion]",0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino\nYo te...,"[G, D, Em, C]","[G, D, C, G]","[7, 2, 16, 0]","[7, 2, 0, 7]","[7, 2, 16, 0]","[7, 2, 0, 7]","[-5, 14, -16]","[-5, -2, 7]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[guitar, bass]",0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,"[D, A, Bm, G]","[D, A, G, A]","[2, 9, 23, 7]","[2, 9, 7, 9]","[-5, 2, 16, 0]","[-5, 2, 0, 2]","[7, 14, -16]","[7, -2, 2]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[guitar, drums]",0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...","[F, G, Am, Em]","[F, G, C, Am]","[5, 7, 21, 16]","[5, 7, 0, 21]","[1, 3, 17, 12]","[1, 3, -4, 17]","[2, 14, -5]","[2, -7, 21]"
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,"[guitar, bass]",0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide\ntodo...,"[A, C#m, F#m, E]","[A, C#m, F#m, E]","[9, 13, 18, 4]","[9, 13, 18, 4]","[1, 5, 10, -4]","[1, 5, 10, -4]","[4, 5, -14]","[4, 5, -14]"
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,"[guitar, piano]",0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl...","[Am, D7, G, C]","[F, E7, Am, G]","[21, 2, 7, 0]","[5, 4, 21, 7]","[13, -6, -1, -8]","[-3, -4, 13, -1]","[-19, 5, -7]","[-1, 17, -14]"


In [150]:
df_vectorial.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [151]:
df_vectorial = df_vectorial.fillna(0)

In [152]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,"[guitar, accordion]",0.0541,0.0000,0,1,102.009,-7.864,Te mando flores que recojo en el camino Yo te ...,"[G, D, Em, C]","[G, D, C, G]","[7, 2, 16, 0]","[7, 2, 0, 7]","[7, 2, 16, 0]","[7, 2, 0, 7]","[-5, 14, -16]","[-5, -2, 7]"
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,"[guitar, bass]",0.0368,0.0000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,"[D, A, Bm, G]","[D, A, G, A]","[2, 9, 23, 7]","[2, 9, 7, 9]","[-5, 2, 16, 0]","[-5, 2, 0, 2]","[7, 14, -16]","[7, -2, 2]"
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,"[guitar, drums]",0.0450,0.0141,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...","[F, G, Am, Em]","[F, G, C, Am]","[5, 7, 21, 16]","[5, 7, 0, 21]","[1, 3, 17, 12]","[1, 3, -4, 17]","[2, 14, -5]","[2, -7, 21]"


In [153]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]","[guitar, bass]",0.0383,0.000055,7,1,103.063,-9.068,En la vida conocí mujer igual a la Flaca Coral...,"[A, D, E, A]","[A, D, E, F#m]","[9, 2, 4, 9]","[9, 2, 4, 18]","[2, -5, -3, 2]","[2, -5, -3, 11]","[-7, 2, 5]","[-7, 2, 14]"


# Backup csv

In [154]:
csv_file = 'mi_archivo_temporal.csv'

# Llamar la función para exportar
export_unique_rows(df_vectorial, csv_file)



/Users/gabrielacaseromonge/Documents/GitHub/song_starter/utils/functions.py:280: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: str(x) if isinstance(x, list) else x)
/Users/gabrielacaseromonge/Documents/GitHub/song_starter/utils/functions.py:286: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)


# Expand columns

In [155]:
# Especificar las columnas que contienen listas (excepto 'embedding')
columns_to_expand = ['instruments', 'chords_1', 'chords_2', 
                      'chords_1_numeric', 'chords_2_numeric', 
                      'normal_chords_1_numeric', 'normal_chords_2_numeric', 
                      'diff_chords_1_numeric', 'diff_chords_2_numeric']

Input:

In [156]:
input_expanded = expand_and_remove_original_columns(input_data, columns_to_expand)

In [157]:
input_expanded

,id,name,artists,chords,speechiness,instrumentalness,key,mode,tempo,loudness,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,0ll9JAIrM3NVBOMl45sxSX,La flaca,Jarabe De Palo,"[A, D, E, A, A, D, E, F#m]",0.0383,0.000055,7,1,103.063,-9.068,...,2,-5,-3,11,-7,2,5,-7,2,14


In [158]:
#elimino las columnas que tengan solo valores nulos
input_expanded = input_expanded.dropna(axis=1, how='all')

Related tracks:

In [159]:
df_vectorial_expanded = expand_and_remove_original_columns(df_vectorial, columns_to_expand)

In [160]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino Yo te ...,...,7,2,0,7,-5,14,-16,-5,-2,7
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,...,-5,2,0,2,7,14,-16,7,-2,2
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...",...,1,3,-4,17,2,14,-5,2,-7,21
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide todo ...,...,1,5,10,-4,4,5,-14,4,5,-14
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl...",...,-3,-4,13,-1,-19,5,-7,-1,17,-14


In [161]:
#elimino las columnas que tengan solo valores nulos
df_vectorial_expanded = df_vectorial_expanded.dropna(axis=1, how='all')

In [162]:
drop_columns_with_many_nulls(df_vectorial_expanded, 15)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino Yo te ...,...,7,2,0,7,-5,14,-16,-5,-2,7
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,...,-5,2,0,2,7,14,-16,7,-2,2
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...",...,1,3,-4,17,2,14,-5,2,-7,21
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide todo ...,...,1,5,10,-4,4,5,-14,4,5,-14
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl...",...,-3,-4,13,-1,-19,5,-7,-1,17,-14


In [163]:
df_vectorial_expanded.head(3)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,0.0541,0.0000,0,1,102.009,-7.864,Te mando flores que recojo en el camino Yo te ...,...,7,2,0,7,-5,14,-16,-5,-2,7
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,0.0368,0.0000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,...,-5,2,0,2,7,14,-16,7,-2,2
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,0.0450,0.0141,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...",...,1,3,-4,17,2,14,-5,2,-7,21


# Create column for the text to create the embeddings

Input:

In [164]:
input_expanded=to_embbed_text_column(input_expanded)

In [165]:
input_expanded['to_embbed_text']

0    name: La flaca artists: Jarabe De Palo chords:...
Name: to_embbed_text, dtype: object

Related songs:

In [166]:
df_vectorial_expanded=to_embbed_text_column(df_vectorial_expanded)

In [167]:
df_vectorial_expanded


,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino Yo te ...,...,2,0,7,-5,14,-16,-5,-2,7,name: Te Mando Flores artists: Fonseca speechi...
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,...,2,0,2,7,14,-16,7,-2,2,name: Me gusta como eres artists: Jarabe De Pa...
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...",...,3,-4,17,2,14,-5,2,-7,21,name: El Duelo artists: La Ley speechiness: 0....
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide todo ...,...,5,10,-4,4,5,-14,4,5,-14,name: La Muralla Verde artists: Los Enanitos V...
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl...",...,-4,13,-1,-19,5,-7,-1,17,-14,name: Canción Del Elegido artists: Silvio Rodr...


# Text for embeddings to chat gpt

In [168]:
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API en OpenAI
OpenAI.api_key = api_key

In [169]:
# Convert text into vectors using embeddings
client = OpenAI(
 api_key=api_key,
)

Input:

In [170]:
input_expanded['embedding'] = input_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [171]:
input_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [172]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino Yo te ...,...,2,0,7,-5,14,-16,-5,-2,7,name: Te Mando Flores artists: Fonseca speechi...
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,...,2,0,2,7,14,-16,7,-2,2,name: Me gusta como eres artists: Jarabe De Pa...
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...",...,3,-4,17,2,14,-5,2,-7,21,name: El Duelo artists: La Ley speechiness: 0....
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide todo ...,...,5,10,-4,4,5,-14,4,5,-14,name: La Muralla Verde artists: Los Enanitos V...
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl...",...,-4,13,-1,-19,5,-7,-1,17,-14,name: Canción Del Elegido artists: Silvio Rodr...


Related songs:

In [173]:
#para cada fila del dataframe, sacar el vector de embedding de la columna "to_embbed_text"
df_vectorial_expanded['embedding'] = df_vectorial_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [174]:
df_vectorial_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [175]:
df_vectorial_expanded = df_vectorial_expanded.rename(columns={'embedding': 'values'})

# Data to pinecone

In [176]:

# Procesar el DataFrame
df_vectorial_expanded = create_metadata_column(df_vectorial_expanded)
df_vectorial_expanded = clean_metadata_column(df_vectorial_expanded)
df_para_pinecone = create_dataframe_for_pinecone(df_vectorial_expanded)
insert_into_pinecone(df_para_pinecone, index)



sending upsert requests:   0%|          | 0/5 [00:00<?, ?it/s]

In [177]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,values,metadata
0,2in1gSZtv2lRbcZUmN9aAl,Te Mando Flores,Fonseca,0.0541,0.000000,0,1,102.009,-7.864,Te mando flores que recojo en el camino Yo te ...,...,0,7,-5,14,-16,-5,-2,7,"[0.04661887139081955, 0.02532416209578514, 0.0...","{'name': 'Te Mando Flores', 'artists': 'Fonsec..."
1,0mlnmHVwY0o096HMTRUsmJ,Me gusta como eres,Jarabe De Palo,0.0368,0.000000,7,1,79.923,-6.206,Como una barca de papel que cuando se moja se ...,...,0,2,7,14,-16,7,-2,2,"[0.028510931879281998, -0.009818550199270248, ...","{'name': 'Me gusta como eres', 'artists': 'Jar..."
2,3XHg42QOqlevRU7jXRkAKk,El Duelo,La Ley,0.0450,0.014100,4,0,107.069,-6.422,"The theme of ""El Duelo"" by La Ley revolves aro...",...,-4,17,2,14,-5,2,-7,21,"[-0.01564365066587925, 0.00896920170634985, 0....","{'name': 'El Duelo', 'artists': 'La Ley', 'spe..."
3,6OKhBvddAlWxxFnjbpilhu,La Muralla Verde,Los Enanitos Verdes,0.0282,0.000012,8,0,129.881,-6.968,Estoy parado sobre la muralla que divide todo ...,...,10,-4,4,5,-14,4,5,-14,"[0.03969547525048256, -0.010846862569451332, -...","{'name': 'La Muralla Verde', 'artists': 'Los E..."
4,33QDwXW1O0hAGDdKRqpqjB,Canción Del Elegido,Silvio Rodríguez,0.0841,0.000001,8,1,152.197,-12.273,"""Canción Del Elegido"" by Silvio Rodríguez expl...",...,13,-1,-19,5,-7,-1,17,-14,"[0.04149605333805084, -0.03992234915494919, -0...","{'name': 'Canción Del Elegido', 'artists': 'Si..."


In [178]:
df_para_pinecone

,id,values,metadata
0,2in1gSZtv2lRbcZUmN9aAl,"[0.04661887139081955, 0.02532416209578514, 0.0...","{'name': 'Te Mando Flores', 'artists': 'Fonsec..."
1,0mlnmHVwY0o096HMTRUsmJ,"[0.028510931879281998, -0.009818550199270248, ...","{'name': 'Me gusta como eres', 'artists': 'Jar..."
2,3XHg42QOqlevRU7jXRkAKk,"[-0.01564365066587925, 0.00896920170634985, 0....","{'name': 'El Duelo', 'artists': 'La Ley', 'spe..."
3,6OKhBvddAlWxxFnjbpilhu,"[0.03969547525048256, -0.010846862569451332, -...","{'name': 'La Muralla Verde', 'artists': 'Los E..."
4,33QDwXW1O0hAGDdKRqpqjB,"[0.04149605333805084, -0.03992234915494919, -0...","{'name': 'Canción Del Elegido', 'artists': 'Si..."


# Query search in pinecone

In [179]:
response = realizar_consulta(client, input_expanded, index, top_k=5)

Resultado de la consulta: {'matches': [{'id': '1rqGuE3jvcvacQk96lfnl6',
              'metadata': {'artists': 'Geese',
                           'chords_1_1': 'G',
                           'chords_1_2': 'D',
                           'chords_1_3': 'Em',
                           'chords_1_4': 'C',
                           'chords_1_numeric_1': 7.0,
                           'chords_1_numeric_2': 2.0,
                           'chords_1_numeric_3': 16.0,
                           'chords_1_numeric_4': 0.0,
                           'chords_2_1': 'Am',
                           'chords_2_2': 'F',
                           'chords_2_3': 'C',
                           'chords_2_4': 'G',
                           'chords_2_numeric_1': 21.0,
                           'chords_2_numeric_2': 5.0,
                           'chords_2_numeric_3': 0.0,
                           'chords_2_numeric_4': 7.0,
                           'diff_chords_1_numeric_1': -5.0,
                 

In [180]:
response_matches = response['matches']

details = []
for match in response_matches:
    details.append(f"Intruments: {match['metadata']['instruments_1']} {match['metadata']['instruments_2']}, Tempo: {match['metadata']['tempo']}, Speechiness: {match['metadata']['speechiness']}, Loudness: {match['metadata']['loudness']}")

In [181]:
details

['Intruments: guitar bass, Tempo: 131.948, Speechiness: 0.0382, Loudness: -7.699',
 'Intruments: piano cello, Tempo: 133.355, Speechiness: 0.033, Loudness: -12.636',
 'Intruments: guitar saxophone, Tempo: 131.685, Speechiness: 0.0469, Loudness: -4.09',
 'Intruments: piano drums, Tempo: 124.357, Speechiness: 0.0678, Loudness: -7.621',
 'Intruments: guitar piano, Tempo: 77.688, Speechiness: 0.0469, Loudness: -18.202']

In [182]:

for match in response_matches:
    print(f"Letra: {match['metadata']['theme']}, \n Score: {match['score']}")

Letra: I’ve been hit by the bus of love That falling brick of you Baby, when there’s tears in your eyes You don’t have to lie, 
 Score: 0.202791825
Letra: Love and learn Take your turn Somewhere there's a feather Falling slowly from the sky, 
 Score: 0.191922888
Letra: The song "El Encuentro" by Biela explores themes of connection, longing, and intimacy. It captures the emotions surrounding a significant meeting between two individuals, emphasizing the beauty and complexity of human relationships. Through its lyrical imagery, it conveys a sense of urgency and the profound impact such encounters can have on one’s life. Overall, the song reflects on the joys and challenges of love and the deep bonds formed through shared experiences., 
 Score: 0.186530888
Letra: Oh-oh-oh-oh Mhmm, mhmm From Bamenda, it's Libianca Oh-oh, 
 Score: 0.185203373
Letra: Mmm, mmm It's a hundred and three in the Valley Blacktop is meltin' on our shoes And I don't mean to make it all about me (Mmm, mmm) But I used

In [183]:
diff_chord_2= input_data['diff_chords_2_numeric'].iloc[0]
diff_chord_1= input_data['diff_chords_1_numeric'].iloc[0]

In [184]:
# Crear el embedding de la nueva query
chords_query = f"A song with a similar progession and distance (diff) in chords to {input_data['chords_1'][0]},y diff chords {diff_chord_2},{diff_chord_1}."
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=chords_query
).data[0].embedding

chords_query = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

chords_query_ = chords_query['matches']



In [185]:
for match in chords_query_:
    print(f"Chords: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}, \n Score: {match['score']}")

Chords: C G Am F, D A Bm G, 
 Score: 0.451347947
Chords: C G Am F, C G F G, 
 Score: 0.451108187
Chords: 2023 Remaster C G Am, F C G F, 
 Score: 0.448859572
Chords: D C G E, D C G A, 
 Score: 0.448275626
Chords: C G Am F, D A Bm G, 
 Score: 0.446885526


In [186]:
chord_wheels=[]
for match in chords_query_:
    chord_wheels.append(f"Chord wheel: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}")

In [187]:
chord_wheels

['Chord wheel: C G Am F, D A Bm G',
 'Chord wheel: C G Am F, C G F G',
 'Chord wheel: 2023 Remaster C G Am, F C G F',
 'Chord wheel: D C G E, D C G A',
 'Chord wheel: C G Am F, D A Bm G']

In [188]:
input_data['theme'][0]

'En la vida conocí mujer igual a la Flaca Coral negro de la Habana, tremendísima mulata Cien libras de piel y hueso, cuarenta kilos de salsa Y en la cara dos soles, que sin palabras hablan Que sin palabras hablan  La Flaca duerme de día, dice que así el hambre engaña '

In [189]:
# Crear el embedding de la nueva query
lyrics_query = f"Give me a similar theme song to {input_data['theme'][0]}"
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=lyrics_query
).data[0].embedding

lyrics_result = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

lyrics_matches = lyrics_result['matches']



In [190]:
lyrics_list = []  # Crear una lista vacía para almacenar los textos

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    lyrics_list.append(f"Lyrics: {match['metadata']['theme']}")

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    print(f"Lyrics: {match['metadata']['theme']} {match['id']} Score: {match['score']}")

Lyrics: Quiero que el aire que toco se transforme en fantasia Quiero que la niña de mis ojos Me diga que siente,me diga que es mia, Me baile y me ria,me coja y se siente, Cuando yo le diga que el vinto la quiere y olé.   5mn6xNwwshCIyCPvafsFqp Score: 0.474218398
Lyrics: Vengo, vengo, vengo de caña vengo, vengo, vengo de son tengo, tengo, tengo en el alma ritmo, ritmo, ritmo y sabor  06fNNrQqwUJ58OPj4u1Otk Score: 0.46688363
Lyrics: Sólo y triste bajo el sol En la playa busco amor Debe haber un lugar para mí Gentes vienen, gentes van 6Ucocvd2YaEJRPmbT4SFgu Score: 0.46378535
Lyrics: Paseo por las huellas, que dejan tus zapatos, me escondo para mirarte en el humo de tu cigarro, bailo con el vuelo de tu falda al caminar... Sonrio por tener, esta amistad contigo, aunque quizas deba llorar  porque nunca seremos mas que amigos  35QHpy90tj0IDKbpHnNsyC Score: 0.457933933
Lyrics: Dispararé a quien quieras por ti, la calle haré cuando quieras por ti. Un perro soy y me hacen ladrar,  no tengo elecc

In [193]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "Eres un asistente experto en escribir canciones. A continuación te proporciono letras de canciones en las que basarte y ruedas de acordes. También información general como instrumentos que usar, tempo y loudness (potencia) de la canción. Después de estos ejemplos, quiero que escribas una nueva canción basada en lo siguiente:"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Especifica el tempo de la canción, las distintas ruedas de acordes a utilizar, qué dos instrumentos podrían ser los principales y cuánta potencia tendría. Básate en: {lyrics_list} .{chord_wheels} . {details}. Básate únicamente en los ejemplos para crear la letra"
        }
      ]
    }
  ],
  temperature=0.5,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format={
    "type": "text"
  }
)

print(response.choices[0].message)

ChatCompletionMessage(content='Para crear una nueva canción, primero estableceré los detalles como el tempo, las ruedas de acordes, los instrumentos principales y la potencia. Luego, escribiré la letra basándome en estos elementos.\n\n**Detalles de la nueva canción:**\n\n- **Tempo:** 100 BPM\n- **Ruedas de acordes:**\n  - Verso: C - G - Am - F\n  - Estribillo: F - C - G - Am\n- **Instrumentos principales:** Guitarra acústica y piano\n- **Potencia:** Media-alta\n\n**Letra de la canción:**\n\n**Verso 1:**\nEn el amanecer, cuando el sol se asoma,  \nLos sueños despiertan, el día se desploma.  \nCaminamos juntos, sin mirar atrás,  \nEn este viaje, no hay marcha atrás.\n\n**Estribillo:**\nPorque en tus ojos, veo el mar,  \nUn horizonte que me invita a volar.  \nCon cada latido, siento la verdad,  \nQue en tus brazos, encontré mi hogar.\n\n**Verso 2:**\nLas estrellas brillan en la noche oscura,  \nSusurran secretos, de amor y locura.  \nNuestros corazones laten al compás,  \nDe una melodía q